In [1]:
# Imports from our QAOA package
from forest_qaoa.qaoa.parameters import StandardParams, ExtendedParams, AbstractParams
from pyquil.paulis import PauliSum, PauliTerm
from pyquil.api import local_qvm, WavefunctionSimulator
from forest_qaoa.qaoa.cost_function import QAOACostFunctionOnWFSim
from scipy.optimize import minimize

In [2]:
hamiltonian = PauliSum.from_compact_str("0.7*Z0*Z1 + 1.2*Z0*Z2 + (-0.5)*Z0")
print("hamiltonian =", hamiltonian)

timesteps = 1

hamiltonian = (0.7+0j)*Z0*Z1 + (1.2+0j)*Z0*Z2 + (-0.5+0j)*Z0


Set up two different sets of extended parameters:

In [3]:
extended1 = ExtendedParams.linear_ramp_from_hamiltonian(hamiltonian, timesteps)
extended2 = ExtendedParams.linear_ramp_from_hamiltonian(hamiltonian, timesteps)

The address of their `x_rotation_angles` always seems to be identical. The same is true sometimes, but not always, for the `z_rotation_angles` and the `zz_rotation_angles`

In [4]:
print(id(extended1.x_rotation_angles))
print(id(extended2.x_rotation_angles))

139774570718832
139774570718832


In [5]:
print(id(extended1.z_rotation_angles))
print(id(extended2.z_rotation_angles))

139775752568304
139774570719312


In [6]:
print(id(extended1.zz_rotation_angles))
print(id(extended2.zz_rotation_angles))

139774570717632
139774650835152


This becomes an issue when we define different sets of parameters in the same notebook, and then an optimiser updates the `x_rotation_angles` of one of the sets: naively we'd think it shouldn't affect the other at all, but since the memory location is the same, it does affect the second set.

## Example

In [7]:
sim = WavefunctionSimulator()

costfn1 = QAOACostFunctionOnWFSim(hamiltonian,
                                  params=extended1,
                                  sim=sim,
                                  scalar_cost_function=True, 
                                  nshots=1,                  
                                  noisy=False,               
                                  enable_logging=False)

costfn2 = QAOACostFunctionOnWFSim(hamiltonian,
                                  params=extended2,
                                  sim=sim,
                                  scalar_cost_function=True, 
                                  nshots=1,                  
                                  noisy=False,               
                                  enable_logging=False)

Print the two sets of `x_rotation_angles` before running the optimiser:

In [8]:
print('extended1 before', extended1.x_rotation_angles)
print('extended2 before', extended2.x_rotation_angles)

extended1 before [[0.35 0.35 0.35]]
extended2 before [[0.35 0.35 0.35]]


In [9]:
res1 = minimize(costfn1, extended1.raw(), tol=1e-3,
                      options={"maxiter": 500})

... and now after running the optimiser:

In [10]:
print('extended1 after', extended1.x_rotation_angles)
print('extended2 after', extended2.x_rotation_angles)

extended1 after [[4.83517474e-05 7.85112595e-01 7.85294732e-01]]
extended2 after [[4.83517474e-05 7.85112595e-01 7.85294732e-01]]


Despite the fact we have not called the optimiser on `costfn2` yet, its associated angles have been modified by the optimiser call on `costfn1`.